In [ ]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression,LinearRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn import metrics, svm
from fast_ml.model_development import train_valid_test_split
import warnings
warnings.filterwarnings("ignore")


In [ ]:
import pandas as pd 
import numpy as np

# datasets
activity_wsdata = pd.read_csv('C:/Users/m/Downloads/data/activity data.csv')
ALEdata = pd.read_csv('C:/Users/m/Downloads/data/ALE in Sensor.csv')
Occupancy = pd.read_csv('C:/Users/m/Downloads/data/Occupancy.csv')
poisioned_data = pd.read_csv('C:/Users/m/Downloads/data/bad_dataset.csv')
powerConsumption = pd.read_csv('C:/Users/m/Downloads//data/bad_power_consumption.csv')

# poisoned data 
target = 'Occupancy'
X_p = poisioned_data.drop(columns=[target])
y_p = poisioned_data[target]

# split data
X_train_p, X_test_p, y_train_p, y_test_p = train_test_split(X_p, y_p, test_size=0.5, random_state=42)
model_train_p, model_test_p, target_train_p, target_test_p = train_test_split(X_train_p, y_train_p, test_size=0.5, random_state=42)
edgeDevice_X_p, edgeData_X_p, edgeDevice_Y_p, edgeData_Y_p = train_test_split(model_train_p, target_train_p, train_size=0.5, random_state=42)
edgeServer1_X_p, edgeServer2_X_p, edgeServer1_Y_p, edgeServer2_Y_p = train_test_split(edgeDevice_X_p, edgeData_X_p, train_size=0.5, random_state=42)

# poisoned data 
target = 'powerConsumption'
X_p = poisioned_data.drop(columns=[target])
y_p = poisioned_data[target]

# split data
X_train_c, X_test_c, y_train_c, y_test_c = train_test_split(X_p, y_p, test_size=0.5, random_state=42)
model_train_c, model_test_c, target_train_c, target_test_c = train_test_split(X_train_c, y_train_c, test_size=0.5, random_state=42)
edgeDevice_X_c, edgeData_X_c, edgeDevice_Y_c, edgeData_Y_c = train_test_split(model_train_c, target_train_c, train_size=0.5, random_state=42)
edgeServer1_X_c, edgeServer2_X_c, edgeServer1_Y_c, edgeServer2_Y_c = train_test_split(edgeDevice_X_c, edgeData_X_c, train_size=0.5, random_state=42)


# clean data 
target = 'Occupancy'
X = Occupancy.drop(columns=[target])
y = Occupancy[target]

# split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)
model_train, model_test, target_train, target_test = train_test_split(X_train, y_train, test_size=0.5, random_state=42)
edgeDevice_X, edgeData_X, edgeDevice_Y, edgeData_Y = train_test_split(model_train, target_train, train_size=0.5, random_state=42)
edgeServer1_X, edgeServer2_X, edgeServer1_Y, edgeServer2_Y = train_test_split(edgeDevice_X, edgeData_X, train_size=0.5, random_state=42)

In [ ]:
import hashlib
import time
import random as rand
import pandas as pd
import numpy as np
import psutil
import matplotlib.pyplot as plt

# Node representing each block
class Node:
    
    def __init__(self, idx, prev_hash, t_stamp, transactions, proof, validator):
        self.idx = idx
        self.prev_hash = prev_hash
        self.t_stamp = t_stamp
        self.transactions = transactions
        self.proof = proof
        self.validator = validator
        self.hash = self.calculate_hash()
    
    def calculate_hash(self):
        node_str = f"{self.idx}{self.prev_hash}{self.t_stamp}{self.transactions}{self.proof}{self.validator}"
        return hashlib.sha256(node_str.encode('utf-8')).hexdigest()


class Blockchain_Network:
    
    def __init__(self):
        self.chain = []
        self.transactions = []
        self.c_node_idx = 0         
        self.validators = ['R_Edge_Server_A', 'R_Edge_Server_B', 'R_Edge_Server_C', 'R_Edge_Server_D', 'R_Edge_Server_E']
        self.add_node()
        self.consensus_threshold = 3 
        self.failed_nodes = []
        
        # Track transaction throughput, validator performance, and block finality times
        self.transaction_times = []
        self.cpu_usages = {validator: [] for validator in self.validators}
        self.transaction_counts = {validator: 0 for validator in self.validators}
        self.node_creation_times = []  
        self.failed_nodes = 0  
        self.successful_nodes = 0  

    def add_node(self):
        # Initial block
        if self.c_node_idx == 0:
            start_node = Node(0, "0", time.time(), [], 'Start', 'System')
            self.chain.append(start_node)
            self.c_node_idx += 1

    # Simulate a transaction creation
    def transaction_session(self, source, receiver, server_id, data, transaction_time):
        transaction = {"AI Vendor": source, 
                       "Edge Device": receiver, 
                       "Edge Server": server_id,
                       "model data": data, 
                       "timestamp": transaction_time}
        self.transactions.append(transaction)

    
    # Simulate the mining process
    def mine_node(self):
        
        start_time = time.time()
        cpu_start = psutil.cpu_percent(interval=None)
        
        validator = rand.choice(self.validators) 
        t_stamp = time.time()
        
        # Simulate proof of authority by generating a random proof
        proof = rand.randint(150, 8500)  
        node = Node(self.c_node_idx, self.chain[-1].hash, t_stamp, self.transactions, proof, validator)

        for transaction in enumerate(self.transactions):
            
            # Check for data integrity and accuracy before adding the node
            integrity_check = self.data_integrity(node.transactions)
            
            if not integrity_check:
                print(f"Node {node.idx} failed the data integrity check. Node discarded.")
                self.failed_blocks += 1  
                continue
           

        end_time = time.time()
        cpu_end = psutil.cpu_percent(interval=None)

        time_taken = end_time - start_time
        cpu_usage = cpu_end - cpu_start
        
        # Track the transaction time and CPU usage
        self.transaction_times.append(time_taken)
        self.cpu_usages[validator].append(cpu_usage)
        self.transaction_counts[validator] += 1
        
        # print(f"Transaction Time Taken: {time_taken:.4f} seconds")
        # print(f"CPU Usage for Transaction: {cpu_usage:.4f}%")
        
        # Simulate validator votes for the block
        votes = self.node_votes(node)
        
        if len(votes) >= self.consensus_threshold: 
            self.chain.append(node)  
            self.c_node_idx += 1
            self.transactions = []
            
            # Node creation time tracker
            node_creation_time = time.time() - start_time
            self.node_creation_times.append(node_creation_time)
            self.successful_nodes += 1  
            
            print(f"Node {node.idx} successfully added to the blockchain.")
        else:          
            print(f"Node {node.idx} failed to reach consensus. Node discarded.")
            self.failed_nodes += 1  # Increment failed block counter
            
        return node

    # Simulate validators voting on the block   
    def node_votes(self, node):
        votes = []
        
        for validator in self.validators:
            votes.append(validator)   
        return votes

    def data_integrity(self, transactions):
        # Accuracy, consistency, data mismatch, and outliers
        for transaction in transactions:
            if isinstance(transaction['model data'], pd.DataFrame):
                missing_values = transaction['model data'].isnull().sum().sum()
                duplicate_values = transaction['model data'].duplicated().sum()
                incorrect_data = 0
                
                for col in transaction['model data'].select_dtypes(include=[np.number]).columns:
                    if not pd.api.types.is_numeric_dtype(transaction['model data'][col]):
                        incorrect_data += 1
                        
                # Check for outliers in the numeric columns of the dataframe
                if self.detect_outliers(transaction['model data']):
                    print(f"Node failed due to outliers in the model data.")
                    return False
                
                if missing_values > 0 or duplicate_values > 0 or incorrect_data > 0:
                    return False
            else:
                if not isinstance(transaction['model data'], (int, float)):
                    return False
        return True
        

    def detect_outliers(self, data):
        numeric_cols = data.select_dtypes(include=[np.number]).columns
        for col in numeric_cols:
            Q1 = data[col].quantile(0.25)
            Q3 = data[col].quantile(0.75)
            IQR = Q3 - Q1
            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR
            
            if ((data[col] < lower_bound) | (data[col] > upper_bound)).any():
                return True 


    def get_last_node(self):
        return self.chain[-1]

    def display_chain(self):
        print(f"Total number of nodes: {len(self.chain)}")
        for node in self.chain:
            print(f"----------------------------")
            print(f"Node {node.idx}: {node.hash} | Validator: {node.validator} | Proof: {node.proof}")

    
    def plot_performance(self):
        # Plot transaction throughput (number of transactions mined by each validator)
        plt.figure(figsize=(10, 8))
        
        # Transaction throughput bar chart
        plt.subplot(4, 1, 1)
        throughput_data = list(self.transaction_counts.values())
        plt.bar(self.transaction_counts.keys(), throughput_data, color='blue')
        plt.title('Transaction Throughput per Validator')
        plt.xlabel('Validators')
        plt.ylabel('Number of Transactions')

        # Validator performance (CPU usage per validator)
        plt.subplot(4, 1, 2)
        cpu_data = [np.mean(cpu_usage) for cpu_usage in self.cpu_usages.values()]
        plt.bar(self.cpu_usages.keys(), cpu_data, color='green')
        plt.title('Validator Performance (Average CPU Usage)')
        plt.xlabel('Validators')
        plt.ylabel('Average CPU Usage (%)')

        # Block finality time line chart
        plt.subplot(4, 1, 3)
        plt.plot(range(1, len(self.node_creation_times) + 1), self.node_creation_times, marker='o', color='orange')
        plt.title('Node Creation Time)
        plt.xlabel('Node Number')
        plt.ylabel('Creation Time (seconds)')

        # Annotate the values on the line chart
        for i, txt in enumerate(self.node_creation_times):
            plt.annotate(f"{txt:.2f}", (i+1, self.node_creation_times[i]), textcoords="offset points", xytext=(0, 5), ha='center', fontsize=8)

        # Fault tolerance pie chart
        plt.subplot(4, 1, 4)
        labels = ['Successful Nodes', 'Failed Nodes']
        sizes = [self.successful_nodes, self.failed_nodes]
        colors = ['lightgreen', 'salmon']
        plt.pie(sizes, labels=labels, colors=colors, autopct='%1.1f%%', startangle=90)
        plt.title('Fault Tolerance (Success vs Failures)')

        # Show the plot
        plt.tight_layout()
        plt.show()


In [ ]:
# Initialize the blockchain
PoA_consensus = Blockchain_Network()

# Simulate creating transactions
    # source: AI Marketplace represented as App_VendorID
    # receiver: edge devices defined as ED_RegionID_DeviceID where ED represents the edge device
    # model data: the AI model parameters
    # model accuracy: the accuracy given by the vendor

# # Display the updated blockchain
# PoA_consensus.display_chain()

# transaction 1
PoA_consensus.transaction_session("App_V01", "ED_R1_AR1", ,"ED_R1", model_train, time.time() )
# Mine another block
PoA_consensus.mine_node()

# transaction 2
PoA_consensus.transaction_session("App_V01", "ED_R2_AR2", "ED_R2", edgeDevice_X, time.time())
# Mine another block
PoA_consensus.mine_node()

# transaction 3
PoA_consensus.transaction_session("App_V03", "ED_R4_AR2", "ED_R4", edgeDevice_X, time.time())
# Mine another block
PoA_consensus.mine_node()

# transaction 4
PoA_consensus.transaction_session("App_V03", "ED_R4_AR2","ED_R4", edgeDevice_X, time.time())
# Mine another block
PoA_consensus.mine_node()

# transaction 5
PoA_consensus.transaction_session("App_V03", "ED_R4_AR2", "ED_R4", edgeDevice_X, time.time())
# Mine another block
PoA_consensus.mine_node()

# transaction 6
PoA_consensus.transaction_session("App_V03", "ED_R3_AR2", "ED_R3", edgeDevice_X_c, time.time())
# Mine another block
PoA_consensus.mine_node()

# transaction 7
PoA_consensus.transaction_session("App_V03", "ED_R1_AR1","ED_R1",  edgeDevice_X, time.time())
# Mine another block
PoA_consensus.mine_node()

# transaction 8
PoA_consensus.transaction_session("App_V04", "ED_R3_AR2", "ED_R3", X_train, time.time())
# Mine another block
PoA_consensus.mine_node()

# transaction 9
PoA_consensus.transaction_session("App_V04", "ED_R2_AR2", "ED_R2", X_train_c, time.time())
# Mine another block
PoA_consensus.mine_node()

# transaction 10
PoA_consensus.transaction_session("App_V02", "ED_R2_AR2", "ED_R2", edgeServer1_X, time.time())
# Mine another block
PoA_consensus.mine_node()

# transaction 11
PoA_consensus.transaction_session("App_V02", "ED_R2_AR2", "ED_R2", edgeDevice_X, time.time())
# Mine another block
PoA_consensus.mine_node()

# transaction 12
PoA_consensus.transaction_session("App_V04", "ED_R3_AR4", "ED_R3", edgeServer1_X, time.time())
# Mine another block
PoA_consensus.mine_node()

# transaction 13
PoA_consensus.transaction_session("App_V03", "ED_R3_AR2", "ED_R3", edgeServer1_X, time.time())
# Mine another block
PoA_consensus.mine_node()

# transaction 14
PoA_consensus.transaction_session("App_V02", "ED_R2_AR1", "ED_R2", model_train, time.time())
# Mine another block
PoA_consensus.mine_node()

# transaction 15
PoA_consensus.transaction_session("App_V01", "ED_R1_AR2", "ED_R1", edgeDevice_X, time.time())
# Mine another block
PoA_consensus.mine_node()

# transaction 16
PoA_consensus.transaction_session("App_V01", "ED_R2_AR4", "ED_R2", X_train, time.time())
# Mine another block
PoA_consensus.mine_node()

# transaction 17
PoA_consensus.transaction_session("App_V03", "ED_R1_AR2", "ED_R1", edgeDevice_X_c, time.time())
# Mine another block
PoA_consensus.mine_node()

# transaction 18
PoA_consensus.transaction_session("App_V02", "ED_R3_AR4", "ED_R3", model_train_p, time.time())
PoA_consensus.mine_node()

# transaction 19
PoA_consensus.transaction_session("App_V01", "ED_R2_AR2", "ED_R2", edgeServer2_X_p, time.time())
# Mine another block
PoA_consensus.mine_node()

# transaction 20
PoA_consensus.transaction_session("App_V04", "ED_R2_AR4", "ED_R2", edgeServer2_X, time.time())
# Mine another block
PoA_consensus.mine_node()

# transaction 21
PoA_consensus.transaction_session("App_V03", "ED_R2_AR5", "ED_R2", edgeData_X, time.time())
# Mine another block
PoA_consensus.mine_node()

# transaction 22
PoA_consensus.transaction_session("App_V02", "ED_R4_AR1","ED_R4", model_train_c, time.time())
# Mine another block
PoA_consensus.mine_node()

# transaction 23
PoA_consensus.transaction_session("App_V03", "ED_R1_AR2", "ED_R1", edgeDevice_X, time.time())
# Mine another block
PoA_consensus.mine_node()

# transaction 24
PoA_consensus.transaction_session("App_V03", "ED_R1_AR1", "ED_R1", model_train_p, time.time())
PoA_consensus.mine_node()

# transaction 25
PoA_consensus.transaction_session("App_V02", "ED_R2_AR2", "ED_R2", edgeDevice_X_p, time.time())
# Mine another block
PoA_consensus.mine_node()

# transaction 26
PoA_consensus.transaction_session("App_V02", "ED_R4_AR1", "ED_R4", model_train, time.time())
# Mine another block
PoA_consensus.mine_node()

# transaction 27
PoA_consensus.transaction_session("App_V03", "ED_R2_AR2", "ED_R2", edgeDevice_X, time.time())
# Mine another block
PoA_consensus.mine_node()

# transaction 28
PoA_consensus.transaction_session("App_V03", "ED_R4_AR2", "ED_R2", edgeDevice_X, time.time())
# Mine another block
PoA_consensus.mine_node()

# transaction 29
PoA_consensus.transaction_session("App_V02", "ED_R4_AR1", "ED_R4", model_train, time.time())
PoA_consensus.mine_node()

# transaction 30
PoA_consensus.transaction_session("App_V03", "ED_R1_AR2", "ED_R1", edgeDevice_X_c, time.time())
# Mine another block
PoA_consensus.mine_node()

# transaction 31
PoA_consensus.transaction_session("App_V03", "ED_R2_AR2", "ED_R2", edgeDevice_X, time.time())
# Mine another block
PoA_consensus.mine_node()

# transaction 32
PoA_consensus.transaction_session("App_V03", "ED_R4_AR2", "ED_R4", edgeDevice_X, time.time())
# Mine another block
PoA_consensus.mine_node()

# transaction 33
PoA_consensus.transaction_session("App_V03", "ED_R1_AR2", "ED_R1", edgeDevice_X_p, time.time())
# Mine another block
PoA_consensus.mine_node()

# transaction 34
PoA_consensus.transaction_session("App_V03", "ED_R3_AR2", "ED_R3", edgeDevice_X, time.time())
# Mine another block
PoA_consensus.mine_node()

# transaction 35
PoA_consensus.transaction_session("App_V03", "ED_R4_AR2", "ED_R3", edgeDevice_X_p, time.time())
# Mine another block
PoA_consensus.mine_node()

# transaction 36
PoA_consensus.transaction_session("App_V03", "ED_R1_AR1", "ED_R1", model_train_p, time.time())
PoA_consensus.mine_node()

# transaction 36
PoA_consensus.transaction_session("App_V02", "ED_R2_AR2", "ED_R2", edgeDevice_X_p, time.time())
# Mine another block
PoA_consensus.mine_node()

# transaction 37
PoA_consensus.transaction_session("App_V03", "ED_R4_AR2", "ED_R4", edgeDevice_X, time.time())
# Mine another block
PoA_consensus.mine_node()

# transaction 38
PoA_consensus.transaction_session("App_V03", "ED_R4_AR2", "ED_R4", edgeDevice_X_c, time.time())
# Mine another block
PoA_consensus.mine_node()

# transaction 39
PoA_consensus.transaction_session("App_V03", "ED_R4_AR2", "ED_R4", edgeDevice_X, time.time())
# Mine another block
PoA_consensus.mine_node()

# transaction 40
PoA_consensus.transaction_session("App_V03", "ED_R4_AR2", "ED_R4", edgeDevice_X, time.time())
# Mine another block
PoA_consensus.mine_node()

# transaction 41
PoA_consensus.transaction_session("App_V03", "ED_R2_AR2", "ED_R2", edgeDevice_X, time.time())
# Mine another block
PoA_consensus.mine_node()

# transaction 42
PoA_consensus.transaction_session("App_V03", "ED_R4_AR2", "ED_R2", edgeDevice_X_c, time.time() )
# Mine another block
PoA_consensus.mine_node()

# transaction 43
PoA_consensus.transaction_session("App_V03", "ED_R1_AR2", "ED_R1", edgeDevice_X, time.time())
# Mine another block
PoA_consensus.mine_node()

# transaction 44
PoA_consensus.transaction_session("App_V03", "ED_R1_AR2", "ED_R1" edgeDevice_X_c, time.time())
# Mine another block
PoA_consensus.mine_node()

# transaction 45
PoA_consensus.transaction_session("App_V03", "ED_R1_AR1", "ED_R1", model_train_p_c, time.time() )
PoA_consensus.mine_node()

# transaction 46
PoA_consensus.transaction_session("App_V02", "ED_R2_AR2", "ED_R2", edgeDevice_X_p, time.time())
# # # Display the updated blockchain
# PoA_consensus.display_chain()

# transaction 47
PoA_consensus.transaction_session("App_V03", "ED_R1_AR1", "ED_R1", model_train_p, time.time())
PoA_consensus.mine_node()

# transaction 48
PoA_consensus.transaction_session("App_V03", "ED_R4_AR2", "ED_R4", edgeDevice_X, time.time())
# Mine another block
PoA_consensus.mine_node()

# transaction 49
PoA_consensus.transaction_session("App_V03", "ED_R4_AR2", "ED_R4", edgeDevice_X, time.time())
# Mine another block
PoA_consensus.mine_node()

# transaction 50
PoA_consensus.transaction_session("App_V03", "ED_R2_AR2", "ED_R2", edgeDevice_X, time.time())
# Mine another block
PoA_consensus.mine_node()

# # transaction 51
# PoA_consensus.transaction_session("App_V03", "ED_R4_AR2", edgeDevice_X, 80)
# # Mine another block
# PoA_consensus.mine_node()

# # transaction 52
# PoA_consensus.transaction_session("App_V02", "ED_R2_AR2", edgeDevice_X_p, 80)
# # # Display the updated blockchain
PoA_consensus.display_chain()
PoA_consensus.plot_performance()